# Movies 3

## Introduction

Une base de données (hébergée dans le cloud AWS) contenant un certain nombre de données cinématographiques vous est fournie pour exploration et mise en pratique des langages SQL et Python.  
Les données de connexion (user, host) sont indiquées dans le brief, le mot de passe nécessaire est précisé en session par le formateur.  


## 1. Description Générale

En utilisant DBeaver et les paramètres de connexion donnés par ailleurs, faire une description détaillée de la base de données fournie. Présenter notamment le diagramme entité - relation (ERD) et les différentes caractéristiques des tables de la base.  Quelles types de renseignement peut-on tirer de cette base de données ?   

<img src="https://cdn.discordapp.com/attachments/778291355473412107/796670836173832212/cc.PNG"/>

### La base de données fournie s'appelle "movies" composée des tables:

- artists: champs: 
                  - id (int)
                  - lastname (varchar(30))
                  - firstname (varchar(30))
                  - birthyear (int)

- countries: champs: 
                  - code (varchar(4))
                  - name (varchar(30))
                  - language (varchar(30))

- movies: champs: 
                  - id (int)
                  - title (varchar(80))
                  - year (int)
                  - directorId (int)
                  - genre (varchar(20))
                  - summary (text)
                  - countrycode  (varchar(4))

- roles: champs: 
                  - movieId (int)
                  - actorId (int)
                  - character (varchar(255))

Grace aux movieId et actorId l'ont peut avoir une liste de film par acteur, et a l'inverse un film contenant chaque acteur ayant participé au film. 

---

## 2. Exploration SQL via python / pandas
Dans cette partie, on s'attache à explorer la base de données en utilisant le language SQL pour les requêtes et le language Python (+ module pandas) pour récupérer les résultats de façon programmatique.  
Vous pouvez utiliser DBeaver ou outil similaire pour préparer / tester vos requêtes mais celles-ci doivent être reportées et appliquées dans du code Python comme présenté dans les quelques examples à suivre.  
*Pour les résultats "longs", un aperçu des quelques premières lignes et le comptage total des résultats est suffisant.*

In [1]:
import os
import pandas as pd

In [2]:
from sqlalchemy import create_engine
import pymysql

In [12]:
# paramètres de connexion à la base de données movies (MySQL sous AWS RDS)
user = os.environ.get('movies3_user')
password = os.environ.get('movies3_psw') 
host = os.environ.get('movies3_host')  
port = 3306
dbname = 'movies'  

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}")

In [4]:
print(user)

student


### Exemples

#### Tables de base

In [5]:
movies = pd.read_sql_query("SELECT * FROM movies LIMIT 10", engine)
movies.head()

,id,title,year,directorId,genre,summary,countrycode
0,11,La Guerre des étoiles,1977,1,Aventure,"Il y a bien longtemps, dans une galaxie très l...",US
1,24,Kill Bill : Volume 1,2003,138,Action,Au cours d'une cérémonie de mariage en plein d...,US
2,28,Apocalypse Now,1979,1776,Drame,L'état-major américain confie au jeune capitai...,US
3,33,Impitoyable,1992,190,Western,"Après avoir été un impitoyable tueur, toujours...",US
4,38,Eternal Sunshine of the Spotless Mind,2004,201,Science-Fiction,Joël et Clémentine ne voient plus que les mauv...,US


In [6]:
artists = pd.read_sql_query("SELECT * FROM artists LIMIT 10", engine)
artists.head()

,id,lastname,firstname,birthyear
0,1,Lucas,George,1944
1,2,Hamill,Mark,1951
2,3,Ford,Harrison,1942
3,4,Fisher,Carrie,1956
4,5,Cushing,Peter,1913


In [7]:
roles = pd.read_sql_query("SELECT * FROM roles LIMIT 10", engine)
roles.head()

,movieId,actorId,character
0,11,2,Luke Skywalker
1,1891,2,Luke Skywalker
2,1892,2,Luke Skywalker
3,181808,2,Luke Skywalker / Dobbu Scay (voice)
4,11,3,Han Solo


In [8]:
countries = pd.read_sql_query("SELECT * FROM countries LIMIT 10", engine)
countries.head()

,code,name,language
0,AU,Australia,
1,BE,Belgium,
2,BS,Bahamas,
3,CA,Canada,
4,CN,China,


#### Autre requête simple
Q00: Nom et année de naissance des artistes nés après 1975 ? Combien sont-ils ?

In [9]:
sql_query_00 = """
    SELECT *
    FROM artists
    WHERE birthyear > 1975
"""

results = pd.read_sql_query(sql_query_00, engine)
results.head(5)

,id,lastname,firstname,birthyear
0,131,Gyllenhaal,Jake,1980
1,205,Dunst,Kirsten,1982
2,225,Holmes,Ashton,1978
3,258,García Bernal,Gael,1978
4,368,Witherspoon,Reese,1976


In [10]:
results.shape

(118, 4)

In [11]:
len(results)

118

### Questions

Q01: Quels sont les films de type 'Drame' ? Combien cela représente de films ?  

In [73]:
sql_query_01 = """
    SELECT genre , title, count(title)
    FROM movies
    WHERE genre = "Drame"
    
"""

results = pd.read_sql_query(sql_query_01, engine)
results.head(5)

,genre,title,count(title)
0,Drame,Apocalypse Now,83


Q02: Quels rôles a joué Bruce Willis et dans quels films (triés par ordre chronologique) ?

In [20]:
sql_query_00 = """
    
"""

results = pd.read_sql_query(sql_query_00, engine)
results.head(5)

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'aa.ff' in 'where clause'")
[SQL: 
    SELECT artists.firstname as ff, artists.lastname as ll
    FROM artists
    INNER JOIN artists as aa ON movies.directorId = aa.id
    WHERE aa.ff = "Bruce" AND aa.ll = "Willis"
    GROUP BY title
    ORDER BY year
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

Q03: Qui est le réalisateur de Memento ?

In [30]:
sql_query_00 = """
    SELECT movies.title, artists.id, artists.lastname, artists.firstname
    FROM artists
    LEFT JOIN movies ON artists.id = movies.directorId
    WHERE movies.title = "Memento";
"""

results = pd.read_sql_query(sql_query_00, engine)
results.head(1)

,title,id,lastname,firstname
0,Memento,525,Nolan,Christopher


Q04: Qui a joué le rôle de Chewbacca et dans quels films ?

In [69]:
sql_query_04 = """
    SELECT title AS "Titre", roles.character AS "Rôle", lastname AS "Nom", firstname AS "Prénom"
    FROM roles
    INNER JOIN artists ON roles.actorId = artists.id
    INNER JOIN movies ON roles.movieId = movies.id
    WHERE roles.character = "Chewbacca";
"""

results = pd.read_sql_query(sql_query_04, engine)
results.head()

,Titre,Rôle,Nom,Prénom
0,La Guerre des étoiles,Chewbacca,Mayhew,Peter
1,L'Empire contre-attaque,Chewbacca,Mayhew,Peter
2,Le Retour du Jedi,Chewbacca,Mayhew,Peter
3,Star Wars : Le Réveil de la Force,Chewbacca,Mayhew,Peter
4,Star Wars : Les Derniers Jedi,Chewbacca,Suotamo,Joonas


Q05: Dans quels films Bruce Willis a-t-il joué le role de John McClane ? 

In [64]:
sql_query_05 = """
    SELECT title, lastname, firstname, roles.character
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Willis" AND firstname = "Bruce" AND     
    roles.character = "John McClane"
"""

results = pd.read_sql_query(sql_query_05, engine)
results.head(5)

,title,lastname,firstname,character
0,Piège de cristal,Willis,Bruce,John McClane
1,Die Hard 4 : Retour en enfer,Willis,Bruce,John McClane
2,Une Journée en enfer,Willis,Bruce,John McClane
3,58 minutes pour vivre,Willis,Bruce,John McClane


Q06: Quels sont les acteurs de 'Sueurs froides' ?

In [66]:
sql_query_06 = """
    SELECT title AS "Titre", lastname AS "Nom", firstname AS "Prénom", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE title = "Sueurs froides";
"""

results = pd.read_sql_query(sql_query_06, engine)
results.head()

,Titre,Nom,Prénom,Rôle
0,Sueurs froides,Stewart,James,Det. John 'Scottie' Ferguson
1,Sueurs froides,Novak,Kim,Madeleine Elster / Judy Barton
2,Sueurs froides,Bel Geddes,Barbara,Marjorie 'Midge' Wood


Q07: Quels sont les films dont le réalisateur est Tim Burton, et l’un des acteurs Jonnhy Depp ?

In [80]:
sql_query_07 = """
SELECT *
    FROM movies
    INNER JOIN (
        SELECT artists.id
        FROM artists
        INNER JOIN movies ON artists.id = movies.directorId
        WHERE artists.lastname = "Burton"
        )
    AS condition ON condition.id = directorId
    INNER JOIN roles ON roles.movieId = movies.id
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE artists.lastname = "Depp"
    ;
"""

results = pd.read_sql_query(sql_query_07, engine)
results.head()

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'condition ON condition.id = directorId\n    INNER JOIN roles ON roles.movieId = m' at line 9")
[SQL: 
SELECT *
    FROM movies
    INNER JOIN (
        SELECT artists.id
        FROM artists
        INNER JOIN movies ON artists.id = movies.directorId
        WHERE artists.lastname = "Burton"
        )
    AS condition ON condition.id = directorId
    INNER JOIN roles ON roles.movieId = movies.id
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE artists.lastname = "Depp"
    ;
]
(Background on this error at: http://sqlalche.me/e/13/f405)

Q08: Quels sont les films dans lesquels a joué Woody Allen ? Donnez aussi le rôle.

In [81]:
sql_query_08 = """
    SELECT title AS "Titre", lastname AS "Nom", firstname AS "Prénom", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE lastname = "Allen" AND firstname = "Woody"
"""

results = pd.read_sql_query(sql_query_08, engine)
results.head()

,Titre,Nom,Prénom,Rôle
0,Scoop,Allen,Woody,Sid Waterman
1,Manhattan,Allen,Woody,Isaac Davis
2,Annie Hall,Allen,Woody,Alvy Singer
3,Maris et femmes,Allen,Woody,Prof. Gabriel 'Gabe' Roth


Q09: Quel metteur en scène a tourné dans ses propres films ? Donnez le nom, le rôle et le titre des films en question.

In [65]:
sql_query_09 = """
    SELECT lastname AS "Nom", firstname AS "Prénom", title AS "Titre", roles.character AS "Rôle"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE movies.directorId = roles.actorId;
"""

results = pd.read_sql_query(sql_query_09, engine)
results.head()

,Nom,Prénom,Titre,Rôle
0,Eastwood,Clint,Impitoyable,Bill Munny
1,Tarantino,Quentin,Reservoir Dogs,Mr. Brown
2,Allen,Woody,Scoop,Sid Waterman
3,Tarantino,Quentin,Pulp Fiction,Jimmie Dimmick
4,Allen,Woody,Manhattan,Isaac Davis


Q10: Quels sont les films de Quentin Tarantino dans lesquels il n’a pas joué 

Q11: Quel metteur en scène a tourné en tant qu’acteur ? Donner le nom, le rôle et le titre des films dans lesquels cet artiste a joué.

,Metteur en scène,Nom en tant qu'acteur,Titre du film,Roles


Q12: Quels sont les films de Hitchcock sans James Stewart

Q13: Quel est la filmographie de James Stewart ?

Q14: Quels sont les films sans rôle (acteurs) ?

Q15: Combien d'acteurs n’ont jamais réalisé de film ? 

Q16: Donnez les noms et prénoms des 10 réalisateurs les plus prolifiques, ainsi que le nombre de films qu’ils ont tournés ?

In [67]:
sql_query_16 = """
    SELECT lastname AS "Nom", firstname AS "Prénom", COUNT(directorId) AS "Nombre de films"
    FROM movies
    INNER JOIN artists ON movies.directorId = artists.id
    GROUP BY directorId
    ORDER BY COUNT(directorId) DESC
    LIMIT 10;
"""

results = pd.read_sql_query(sql_query_16, engine)
results.head()

,Nom,Prénom,Nombre de films
0,Spielberg,Steven,13
1,Hitchcock,Alfred,10
2,Allen,Woody,8
3,Tarantino,Quentin,7
4,Kubrick,Stanley,7


Q17: Donnez les noms et prénoms des réalisateurs qui ont tourné au moins deux films. Combien sont-ils ?

Q18: Dans quels films le réalisateur a-t-il le même prénom que l’un des interprètes ? (titre, nom du réalisateur, nom de l’interprète). Le réalisateur et l’interprète ne doivent pas être la même personne.

Q19: Quels sont les 10 pays produisant le plus de films ?

In [78]:
sql_query_19 = """
    SELECT countrycode, count(countrycode)
    FROM movies
    GROUP BY countrycode
    ORDER BY count(countrycode) DESC
    LIMIT 10
    
"""

results = pd.read_sql_query(sql_query_19, engine)
results.head(10)

,countrycode,count(countrycode)
0,US,131
1,FR,39
2,GB,21
3,CA,6
4,DE,5
5,IT,5
6,JP,4
7,AU,3
8,SE,3
9,BE,2


Q20: Quels sont les 5 acteurs ayant le plus joué dans des films français ?

In [68]:
sql_query_20 = """
    SELECT lastname AS "Nom", firstname AS "Prénom", COUNT(actorId) AS "Nombre de films"
    FROM movies
    INNER JOIN roles ON movies.id = roles.movieId
    INNER JOIN artists ON roles.actorId = artists.id
    WHERE countrycode = "FR"
    GROUP BY actorId
    ORDER BY COUNT(actorId) DESC
    LIMIT 5;
"""

results = pd.read_sql_query(sql_query_20, engine)
results.head()

,Nom,Prénom,Nombre de films
0,Deneuve,Catherine,3
1,Belmondo,Jean-Paul,3
2,Piccoli,Michel,3
3,Fresnay,Pierre,2
4,Arestrup,Niels,2


---

## 3. Expression libre

A vous de jouer ! Tentez quelques requêtes de votre choix (avec pourquoi pas quelques graphiques illustratifs en utilisant Matplotlib...)